In [ ]:



import cv2
import numpy as np
import imageio
import os
import pandas as pd
from abc import ABC, abstractmethod
from typing import Dict, List, Optional, Tuple


class Config:
    """Configuration class for frame restoration parameters"""
    CORRUPTED_INDICES = [0, 4, 9, 14, 18, 23, 28, 32, 37, 42]
    FRAME_SIZE = (1000, 1000, 3)
    ROTATION_FRAMES = {4: 135, 37: 60}
    SALT_FRAMES = [23]
    PEPPER_FRAMES = [28]
    BLUR_FRAMES = [33]
    POSITION_SWAP_FRAMES = {14: 23, 23: 14}
    RED_NOISE_FRAMES = []
    LAST_FRAME = [42]
    FIRST_FRAME = [0]
    REPLACE_FRAMES = {9: 10, 32: 33}


class FrameExtractor:
    """Handles extraction of frames from GIF files"""
    
    def __init__(self, gif_path: str, output_dir: str = 'frames'):
        self.gif_path = gif_path
        self.output_dir = output_dir
    
    def extract_all_frames(self) -> int:
        """Extract all frames from GIF and save as individual PNG files"""
        os.makedirs(self.output_dir, exist_ok=True)
        
        print("=" * 70)
        print("EXTRACTING FRAMES FROM GIF")
        print("=" * 70)
        
        frames = imageio.mimread(self.gif_path)
        print(f"Total frames found: {len(frames)}")
        
        for i, frame in enumerate(frames):
            frame_bgr = cv2.cvtColor(frame, cv2.COLOR_RGB2BGR)
            frame_path = os.path.join(self.output_dir, f'frame_{i}.png')
            cv2.imwrite(frame_path, frame_bgr)
            
            if i in Config.CORRUPTED_INDICES:
                corruption_type = CorruptionTypeDetector.get_corruption_type(i)
                print(f"  Saved: frame_{i}.png [CORRUPTED - {corruption_type}]")
            else:
                print(f"  Saved: frame_{i}.png")
        
        print("=" * 70)
        print(f"✓ All {len(frames)} frames extracted to '{self.output_dir}/' directory")
        print("=" * 70)
        
        return len(frames)


class CorruptionTypeDetector:
    """Detects and identifies types of frame corruption"""
    
    @staticmethod
    def get_corruption_type(frame_idx: int) -> str:
        """Identify corruption type for a frame"""
        if frame_idx in Config.ROTATION_FRAMES:
            return f"ROTATION ({Config.ROTATION_FRAMES[frame_idx]}°)"
        elif frame_idx == 23:
            return "SWAPPED WITH 14 + SALT NOISE"
        elif frame_idx == 14:
            return "SWAPPED WITH 23"
        elif frame_idx in Config.PEPPER_FRAMES:
            return "PEPPER NOISE"
        elif frame_idx in Config.REPLACE_FRAMES:
            return f"REPLACE WITH FRAME {Config.REPLACE_FRAMES[frame_idx]}"
        elif frame_idx in Config.BLUR_FRAMES:
            return "BLUR"
        elif frame_idx in Config.LAST_FRAME:
            return "COPY PREVIOUS"
        elif frame_idx in Config.FIRST_FRAME:
            return "SPECIAL - FOREGROUND + BACKGROUND"
        else:
            return "FOREGROUND/BACKGROUND"


class NoiseDetector:
    """Detects various types of noise in images"""
    
    @staticmethod
    def detect_salt_pepper_noise(img: np.ndarray, 
                                 threshold_low: int = 10, 
                                 threshold_high: int = 245) -> np.ndarray:
        """Detect salt (white) and pepper (black) noise pixels"""
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        pepper_mask = gray < threshold_low
        salt_mask = gray > threshold_high
        noise_mask = np.logical_or(salt_mask, pepper_mask)
        return noise_mask


class ImageProcessor:
    """Basic image processing operations"""
    
    @staticmethod
    def sharpen_image(img: np.ndarray) -> np.ndarray:
        """Apply sharpening filter to image"""
        kernel = np.array([[-1, -1, -1],
                          [-1,  9, -1],
                          [-1, -1, -1]])
        sharpened = cv2.filter2D(img, -1, kernel)
        return sharpened
    
    @staticmethod
    def average_frames(frame1: np.ndarray, frame2: np.ndarray) -> np.ndarray:
        """Average two frames with equal weights"""
        return cv2.addWeighted(frame1, 0.5, frame2, 0.5, 0)


class MaskCreator:
    """Handles interactive mask creation"""
    
    def __init__(self):
        self.drawing = False
        self.erasing = False
        self.mask = None
        self.brush_size = 15
        self.current_image = None
    
    def create_mask_interactive(self, image: np.ndarray, window_title: str) -> Optional[np.ndarray]:
        """Create foreground mask interactively"""
        if image is None or image.size == 0:
            print("Error: Invalid image provided!")
            return None
            
        self.mask = np.zeros((image.shape[0], image.shape[1], 3), dtype=np.uint8)
        self.current_image = image.copy()
        
        cv2.namedWindow(window_title, cv2.WINDOW_NORMAL | cv2.WINDOW_KEEPRATIO)
        cv2.resizeWindow(window_title, 800, 800)
        cv2.setMouseCallback(window_title, self._mouse_callback)
        
        self._print_instructions(window_title)
        
        display_image = self._create_display_image()
        cv2.imshow(window_title, display_image)
        cv2.waitKey(100)
        
        print("\n⚠️  WINDOW OPENED! If you don't see it, check your taskbar.")
        print("    Click on the window to activate it, then start drawing.\n")
        
        while True:
            display_image = self._create_display_image()
            cv2.imshow(window_title, display_image)
            
            key = cv2.waitKey(10) & 0xFF
            
            if key == ord('+') or key == ord('='):
                self.brush_size = min(self.brush_size + 5, 100)
                print(f"✓ Brush size: {self.brush_size}")
            elif key == ord('-') and self.brush_size > 1:
                self.brush_size = max(self.brush_size - 5, 1)
                print(f"✓ Brush size: {self.brush_size}")
            elif key == ord('c'):
                self.mask = np.zeros((image.shape[0], image.shape[1], 3), dtype=np.uint8)
                print("✓ Mask cleared!")
            elif key == ord('q'):
                print("❌ Mask creation cancelled by user")
                cv2.destroyAllWindows()
                return None
            elif key == ord('s'):
                print("✅ Mask saved!")
                break
            elif key == 27:
                print("❌ Mask creation cancelled (ESC pressed)")
                cv2.destroyAllWindows()
                return None
        
        cv2.destroyAllWindows()
        cv2.waitKey(100)
        return self.mask
    
    def _mouse_callback(self, event, x, y, flags, param):
        """Handle mouse events for drawing"""
        if event == cv2.EVENT_LBUTTONDOWN:
            self.drawing = True
            self.erasing = False
            cv2.circle(self.mask, (x, y), self.brush_size, (255, 255, 255), -1)
        elif event == cv2.EVENT_RBUTTONDOWN:
            self.erasing = True
            self.drawing = False
            cv2.circle(self.mask, (x, y), self.brush_size, (0, 0, 0), -1)
        elif event == cv2.EVENT_MOUSEMOVE:
            if self.drawing:
                cv2.circle(self.mask, (x, y), self.brush_size, (255, 255, 255), -1)
            elif self.erasing:
                cv2.circle(self.mask, (x, y), self.brush_size, (0, 0, 0), -1)
        elif event == cv2.EVENT_LBUTTONUP:
            self.drawing = False
        elif event == cv2.EVENT_RBUTTONUP:
            self.erasing = False
    
    def _create_display_image(self) -> np.ndarray:
        """Create display image with mask overlay"""
        mask_gray = cv2.cvtColor(self.mask, cv2.COLOR_BGR2GRAY)
        overlay_green = np.zeros_like(self.current_image, dtype=np.uint8)
        overlay_green[mask_gray == 255] = (0, 255, 0)
        display_image = cv2.addWeighted(self.current_image, 0.7, overlay_green, 0.3, 0)
        
        h, w = display_image.shape[:2]
        cv2.rectangle(display_image, (5, 5), (w-5, 120), (0, 0, 0), -1)
        cv2.rectangle(display_image, (5, 5), (w-5, 120), (0, 255, 255), 2)
        
        cv2.putText(display_image, f"Brush Size: {self.brush_size}", (15, 35),
                   cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 255), 2)
        
        status = "DRAWING (Left Mouse)" if self.drawing else ("ERASING (Right Mouse)" if self.erasing else "Ready - Click to Start")
        cv2.putText(display_image, status, (15, 70),
                   cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0) if self.drawing else ((255, 100, 100) if self.erasing else (255, 255, 255)), 2)
        
        cv2.putText(display_image, "Press 's' to SAVE | 'q' or ESC to QUIT", (15, 105),
                   cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 1)
        
        return display_image
    
    @staticmethod
    def _print_instructions(window_title: str):
        """Print user instructions"""
        print("\n" + "=" * 80)
        print(f"🖌  NOW EDITING: {window_title}")
        print("=" * 80)
        print("INSTRUCTIONS:")
        print("  ✏️  LEFT CLICK & DRAG   : Paint Foreground (Person) - GREEN")
        print("  🧹 RIGHT CLICK & DRAG  : Erase / Mark Background")
        print("  ➕ '+' or '='          : Increase brush size")
        print("  ➖ '-'                 : Decrease brush size")
        print("  🔄 'c'                 : Clear entire mask")
        print("  💾 's'                 : SAVE mask and continue")
        print("  ❌ 'q' or ESC          : QUIT without saving")
        print("=" * 80)


class NeighborFrameManager:
    """Manages neighbor frame operations"""
    
    def __init__(self, frames_dir: str):
        self.frames_dir = frames_dir
    
    def get_neighbor_indices(self, frame_idx: int) -> Tuple[Optional[int], Optional[int]]:
        """Get previous and next frame indices, skipping corrupted ones"""
        prev_idx = frame_idx - 1
        next_idx = frame_idx + 1
        
        while prev_idx in Config.CORRUPTED_INDICES and prev_idx >= 0:
            prev_idx -= 1
        
        while next_idx in Config.CORRUPTED_INDICES and next_idx <= 42:
            next_idx += 1
        
        if prev_idx < 0:
            prev_idx = None
        if next_idx > 42:
            next_idx = None
        
        return prev_idx, next_idx
    
    def load_neighbor_frames(self, prev_idx: Optional[int], 
                           next_idx: Optional[int]) -> Tuple[Optional[np.ndarray], Optional[np.ndarray]]:
        """Load neighbor frames, handling missing frames"""
        img_before = None
        img_after = None
        
        if prev_idx is not None:
            img_before = cv2.imread(os.path.join(self.frames_dir, f'frame_{prev_idx}.png'))
        
        if next_idx is not None:
            img_after = cv2.imread(os.path.join(self.frames_dir, f'frame_{next_idx}.png'))
        
        return img_before, img_after


class FrameRestorer(ABC):
    """Abstract base class for frame restoration strategies"""
    
    def __init__(self, frames_dir: str, masks_dir: str):
        self.frames_dir = frames_dir
        self.masks_dir = masks_dir
        self.neighbor_manager = NeighborFrameManager(frames_dir)
    
    @abstractmethod
    def restore(self, frame_idx: int) -> Optional[np.ndarray]:
        """Restore a corrupted frame"""
        pass
    
    def load_frame(self, frame_idx: int) -> np.ndarray:
        """Load a frame by index"""
        return cv2.imread(os.path.join(self.frames_dir, f'frame_{frame_idx}.png'))
    
    def save_restored_frame(self, frame_idx: int, restored: np.ndarray):
        """Save restored frame"""
        output_path = os.path.join(self.frames_dir, f'frame_{frame_idx}_restored.png')
        cv2.imwrite(output_path, restored)
        print(f"\n✅ Frame {frame_idx} restoration complete!")
        print(f"   Saved as: {output_path}")


class RotationRestorer(FrameRestorer):
    """Restores frames with rotation corruption"""
    
    def restore(self, frame_idx: int) -> Optional[np.ndarray]:
        angle = Config.ROTATION_FRAMES[frame_idx]
        print(f"\nRestoring rotation frame {frame_idx} (angle: {angle}°)")
        
        img_current = self.load_frame(frame_idx)
        prev_idx, next_idx = self.neighbor_manager.get_neighbor_indices(frame_idx)
        img_before, img_after = self.neighbor_manager.load_neighbor_frames(prev_idx, next_idx)
        
        height, width = img_current.shape[:2]
        center = (width // 2, height // 2)
        rotation_matrix = cv2.getRotationMatrix2D(center, -angle, 1.0)
        rotated = cv2.warpAffine(img_current, rotation_matrix, (width, height))
        
        gray = cv2.cvtColor(rotated, cv2.COLOR_BGR2GRAY)
        _, black_mask = cv2.threshold(gray, 1, 255, cv2.THRESH_BINARY_INV)
        
        avg_background = ImageProcessor.average_frames(img_before, img_after)
        
        black_mask_3ch = cv2.cvtColor(black_mask, cv2.COLOR_GRAY2BGR)
        restored = np.where(black_mask_3ch == 255, avg_background, rotated)
        
        return restored


class NoiseRestorer(FrameRestorer):
    """Restores frames with salt and pepper noise"""
    
    def restore(self, frame_idx: int) -> Optional[np.ndarray]:
        print(f"\nRestoring salt/pepper noise frame {frame_idx}")
        
        img_current = self.load_frame(frame_idx)
        noise_mask = NoiseDetector.detect_salt_pepper_noise(img_current)
        restored = img_current.copy()
        
        for c in range(3):
            channel = img_current[:, :, c]
            median_filtered = cv2.medianBlur(channel, 3)
            restored[:, :, c] = np.where(noise_mask, median_filtered, channel)
        
        print(f"  Replaced {np.sum(noise_mask)} noise pixels")
        return restored


class SwapRestorer(FrameRestorer):
    """Restores frames with position swap corruption"""
    
    def restore(self, frame_idx: int) -> Optional[np.ndarray]:
        print(f"\nRestoring swapped frame {frame_idx}")
        
        if frame_idx == 14:
            img_23 = self.load_frame(23)
            restored_23_path = os.path.join(self.frames_dir, 'frame_23_restored.png')
            
            if os.path.exists(restored_23_path):
                print(f"  Using already restored frame 23")
                return cv2.imread(restored_23_path)
            else:
                print(f"  Using frame 23 content")
                return img_23
        
        elif frame_idx == 23:
            noise_restorer = NoiseRestorer(self.frames_dir, self.masks_dir)
            denoised = noise_restorer.restore(23)
            print(f"  Removed salt noise and using correct content")
            return denoised


class ReplaceRestorer(FrameRestorer):
    """Restores frames by replacing with another frame"""
    
    def restore(self, frame_idx: int) -> Optional[np.ndarray]:
        replace_with = Config.REPLACE_FRAMES[frame_idx]
        restored = self.load_frame(replace_with)
        print(f"Replaced with frame {replace_with}")
        return restored


class CopyRestorer(FrameRestorer):
    """Restores frames by copying from previous frame"""
    
    def restore(self, frame_idx: int) -> Optional[np.ndarray]:
        prev_idx, _ = self.neighbor_manager.get_neighbor_indices(frame_idx)
        restored = self.load_frame(prev_idx)
        print(f"Copied from frame {prev_idx}")
        return restored


class MaskBasedRestorer(FrameRestorer):
    """Restores frames using foreground/background masking"""
    
    def __init__(self, frames_dir: str, masks_dir: str):
        super().__init__(frames_dir, masks_dir)
        self.mask_creator = MaskCreator()
    
    def restore(self, frame_idx: int) -> Optional[np.ndarray]:
        prev_idx, next_idx = self.neighbor_manager.get_neighbor_indices(frame_idx)
        img_before, img_after = self.neighbor_manager.load_neighbor_frames(prev_idx, next_idx)
        img_current = self.load_frame(frame_idx)
        
        restored_background = self._get_background(img_before, img_after, prev_idx, next_idx)
        if restored_background is None:
            return None
        
        fg_mask = self._get_or_create_mask(frame_idx, img_current)
        if fg_mask is None:
            return None
        
        foreground = self._process_foreground(frame_idx, img_current)
        
        restored = np.where(fg_mask == 255, foreground, restored_background)
        return restored
    
    def _get_background(self, img_before, img_after, prev_idx, next_idx):
        """Determine background from neighbor frames"""
        if img_before is None and img_after is not None:
            print(f"Only next frame {next_idx} available, using it for background")
            return img_after
        elif img_after is None and img_before is not None:
            print(f"Only previous frame {prev_idx} available, using it for background")
            return img_before
        elif img_before is not None and img_after is not None:
            print(f"Using neighbors: frame_{prev_idx} and frame_{next_idx}")
            return ImageProcessor.average_frames(img_before, img_after)
        else:
            print("Error: No neighbor frames available!")
            return None
    
    def _get_or_create_mask(self, frame_idx: int, img_current: np.ndarray) -> Optional[np.ndarray]:
        """Get existing mask or create new one"""
        mask_path = os.path.join(self.masks_dir, f'mask_frame_{frame_idx}.png')
        
        if os.path.exists(mask_path):
            print(f"Found existing mask: {mask_path}")
            return cv2.imread(mask_path)
        else:
            print("No existing mask found. Please create one...")
            fg_mask = self.mask_creator.create_mask_interactive(
                img_current, f"Mark Person in Frame {frame_idx}")
            
            if fg_mask is None:
                print("Operation cancelled by user.")
                return None
            
            os.makedirs(self.masks_dir, exist_ok=True)
            cv2.imwrite(mask_path, fg_mask)
            print(f"✓ Mask saved: {mask_path}")
            return fg_mask
    
    def _process_foreground(self, frame_idx: int, img_current: np.ndarray) -> np.ndarray:
        """Process foreground, applying sharpening if needed"""
        if frame_idx in Config.BLUR_FRAMES:
            print("Applying sharpening to foreground...")
            return ImageProcessor.sharpen_image(img_current)
        return img_current


class SpecialFrame0Restorer(FrameRestorer):
    """Special restoration for frame 0"""
    
    def __init__(self, frames_dir: str, masks_dir: str):
        super().__init__(frames_dir, masks_dir)
        self.mask_creator = MaskCreator()
    
    def restore(self, frame_idx: int) -> Optional[np.ndarray]:
        print("\nRestoring Frame 0 - Special Case")
        print("Need to mark person in BOTH frame 0 and frame 1")
        
        img_frame_0 = self.load_frame(0)
        img_frame_1 = self.load_frame(1)
        
        fg_mask_0 = self._get_or_create_mask(0, img_frame_0, "Mark Person in Frame 0 (to KEEP)")
        if fg_mask_0 is None:
            return None
        
        fg_mask_1 = self._get_or_create_mask(1, img_frame_1, "Mark Person in Frame 1 (to REMOVE)")
        if fg_mask_1 is None:
            return None
        
        fg_mask_0_gray = cv2.cvtColor(fg_mask_0, cv2.COLOR_BGR2GRAY)
        fg_mask_1_gray = cv2.cvtColor(fg_mask_1, cv2.COLOR_BGR2GRAY)
        
        restored = img_frame_1.copy()
        restored = np.where(fg_mask_1_gray[:, :, np.newaxis] == 255, img_frame_0, restored)
        restored = np.where(fg_mask_0_gray[:, :, np.newaxis] == 255, img_frame_0, restored)
        
        print("✓ Combined: Person from frame 0 + Background from frame 1")
        return restored
    
    def _get_or_create_mask(self, frame_idx: int, image: np.ndarray, 
                           window_title: str) -> Optional[np.ndarray]:
        """Get existing mask or create new one"""
        mask_path = os.path.join(self.masks_dir, f'mask_frame_{frame_idx}.png')
        
        if os.path.exists(mask_path):
            print(f"Found existing mask for frame {frame_idx}")
            return cv2.imread(mask_path)
        else:
            print(f"\n--- Step: {window_title} ---")
            fg_mask = self.mask_creator.create_mask_interactive(image, window_title)
            
            if fg_mask is None:
                return None
            
            os.makedirs(self.masks_dir, exist_ok=True)
            cv2.imwrite(mask_path, fg_mask)
            print(f"✓ Mask saved: {mask_path}")
            return fg_mask


class ManualSegmentationWorkflow:
    """Workflow specifically for manual segmentation of any frame"""
    
    def __init__(self, frames_dir: str = 'frames', masks_dir: str = 'masks'):
        self.frames_dir = frames_dir
        self.masks_dir = masks_dir
        self.mask_creator = MaskCreator()
        
    def segment_frame(self, frame_idx: int, force_new: bool = False):
        """Manually segment a specific frame"""
        print(f"\n{'='*70}")
        print(f"MANUAL SEGMENTATION - Frame {frame_idx}")
        print(f"{'='*70}")
        
        frame_path = os.path.join(self.frames_dir, f'frame_{frame_idx}.png')
        if not os.path.exists(frame_path):
            print(f"❌ Error: Frame {frame_idx} not found at {frame_path}")
            return None
            
        img = cv2.imread(frame_path)
        print(f"✓ Loaded frame: {frame_path}")
        
        mask_path = os.path.join(self.masks_dir, f'mask_frame_{frame_idx}.png')
        
        if os.path.exists(mask_path) and not force_new:
            print(f"\n⚠️  Existing mask found: {mask_path}")
            print("   Use force_new=True to create a new mask")
            response = input("   Load existing mask? (y/n): ").lower()
            if response == 'y':
                mask = cv2.imread(mask_path)
                print("✓ Loaded existing mask")
                self._preview_mask(img, mask, frame_idx)
                return mask
        
        print("\n🖌️  Opening interactive masking tool...")
        mask = self.mask_creator.create_mask_interactive(
            img, f"Segment Person - Frame {frame_idx}"
        )
        
        if mask is None:
            print("❌ Segmentation cancelled")
            return None
        
        os.makedirs(self.masks_dir, exist_ok=True)
        cv2.imwrite(mask_path, mask)
        print(f"\n✅ Mask saved: {mask_path}")
        
        self._preview_mask(img, mask, frame_idx)
        
        return mask
    
    def _preview_mask(self, img: np.ndarray, mask: np.ndarray, frame_idx: int):
        """Preview the masked result"""
        mask_gray = cv2.cvtColor(mask, cv2.COLOR_BGR2GRAY)
        
        foreground = np.where(mask_gray[:, :, np.newaxis] == 255, img, 0)
        overlay = cv2.addWeighted(img, 0.5, mask, 0.5, 0)
        
        combined = np.hstack([img, overlay, foreground])
        
        cv2.namedWindow(f'Frame {frame_idx} - Original | Overlay | Extracted', cv2.WINDOW_NORMAL)
        cv2.resizeWindow(f'Frame {frame_idx} - Original | Overlay | Extracted', 1800, 600)
        cv2.imshow(f'Frame {frame_idx} - Original | Overlay | Extracted', combined)
        
        print("\n📺 Preview: Original | Mask Overlay | Extracted Foreground")
        print("   Press any key to close...")
        cv2.waitKey(0)
        cv2.destroyAllWindows()
    
    def segment_multiple_frames(self, frame_indices: List[int]):
        """Segment multiple frames in sequence"""
        print(f"\n{'='*70}")
        print(f"BATCH SEGMENTATION - {len(frame_indices)} frames")
        print(f"{'='*70}")
        print(f"Frames to segment: {frame_indices}\n")
        
        completed = []
        skipped = []
        
        for idx in frame_indices:
            print(f"\n[{frame_indices.index(idx)+1}/{len(frame_indices)}]")
            mask = self.segment_frame(idx)
            
            if mask is not None:
                completed.append(idx)
            else:
                skipped.append(idx)
        
        print(f"\n{'='*70}")
        print(f"✅ Completed: {len(completed)} frames - {completed}")
        if skipped:
            print(f"⏭️  Skipped: {len(skipped)} frames - {skipped}")
        print(f"{'='*70}")
    
    def list_frames_needing_masks(self) -> List[int]:
        """List which corrupted frames don't have masks yet"""
        needs_masks = []
        
        print(f"\n{'='*70}")
        print("MASK STATUS FOR CORRUPTED FRAMES")
        print(f"{'='*70}")
        print(f"{'Frame':<10} {'Has Mask?':<15} {'Corruption Type':<30}")
        print("-"*70)
        
        for idx in Config.CORRUPTED_INDICES:
            mask_path = os.path.join(self.masks_dir, f'mask_frame_{idx}.png')
            has_mask = os.path.exists(mask_path)
            corruption = CorruptionTypeDetector.get_corruption_type(idx)
            
            status = "✓ Yes" if has_mask else "✗ No"
            print(f"{idx:<10} {status:<15} {corruption:<30}")
            
            if not has_mask:
                needs_masks.append(idx)
        
        print("-"*70)
        print(f"\nFrames needing masks: {needs_masks if needs_masks else 'None - all done!'}")
        print(f"{'='*70}")
        
        return needs_masks


class RestorationFactory:
    """Factory class to create appropriate restorer for each frame"""
    
    def __init__(self, frames_dir: str, masks_dir: str):
        self.frames_dir = frames_dir
        self.masks_dir = masks_dir
    
    def get_restorer(self, frame_idx: int) -> FrameRestorer:
        """Get appropriate restorer for frame based on corruption type"""
        if frame_idx == 0:
            return SpecialFrame0Restorer(self.frames_dir, self.masks_dir)
        elif frame_idx in Config.ROTATION_FRAMES:
            return RotationRestorer(self.frames_dir, self.masks_dir)
        elif frame_idx in Config.POSITION_SWAP_FRAMES:
            return SwapRestorer(self.frames_dir, self.masks_dir)
        elif frame_idx in Config.SALT_FRAMES or frame_idx in Config.PEPPER_FRAMES:
            if frame_idx not in Config.POSITION_SWAP_FRAMES:
                return NoiseRestorer(self.frames_dir, self.masks_dir)
        elif frame_idx in Config.REPLACE_FRAMES:
            return ReplaceRestorer(self.frames_dir, self.masks_dir)
        elif frame_idx in Config.LAST_FRAME:
            return CopyRestorer(self.frames_dir, self.masks_dir)
        
        return MaskBasedRestorer(self.frames_dir, self.masks_dir)


class RestorationManager:
    """Manages the complete restoration process"""
    
    def __init__(self, frames_dir: str = 'frames', masks_dir: str = 'masks'):
        self.frames_dir = frames_dir
        self.masks_dir = masks_dir
        self.factory = RestorationFactory(frames_dir, masks_dir)
    
    def restore_all_corrupted_frames(self) -> Dict[int, np.ndarray]:
        """Restore all corrupted frames"""
        print("\n" + "=" * 70)
        print("RESTORING ALL CORRUPTED FRAMES")
        print("=" * 70)
        print(f"Corrupted frames to restore: {Config.CORRUPTED_INDICES}")
        
        self._print_restoration_plan()
        
        restored_frames = {}
        
        for idx in Config.CORRUPTED_INDICES:
            print(f"\n{'#' * 70}")
            print(f"# Processing Frame {idx} ({Config.CORRUPTED_INDICES.index(idx) + 1}/{len(Config.CORRUPTED_INDICES)})")
            print(f"{'#' * 70}")
            
            restored = self._restore_single_frame(idx)
            
            if restored is not None:
                restored_frames[idx] = restored
                print(f"✓ Frame {idx} completed")
            else:
                print(f"✗ Frame {idx} skipped")
        
        print("\n" + "=" * 70)
        print(f"✓ RESTORATION COMPLETE!")
        print(f"  Restored {len(restored_frames)}/{len(Config.CORRUPTED_INDICES)} frames")
        print("=" * 70)
        
        return restored_frames
    
    def _restore_single_frame(self, frame_idx: int) -> Optional[np.ndarray]:
        """Restore a single frame"""
        print("\n" + "=" * 70)
        print(f"RESTORING FRAME {frame_idx} - {CorruptionTypeDetector.get_corruption_type(frame_idx)}")
        print("=" * 70)
        
        restorer = self.factory.get_restorer(frame_idx)
        restored = restorer.restore(frame_idx)
        
        if restored is not None:
            restorer.save_restored_frame(frame_idx, restored)
            self._display_result(frame_idx, restored)
        
        return restored
    
    def _display_result(self, frame_idx: int, restored: np.ndarray):
        """Display restoration result"""
        cv2.namedWindow(f'Final Restored Frame {frame_idx}', cv2.WINDOW_NORMAL)
        cv2.imshow(f'Final Restored Frame {frame_idx}', restored)
        print("\nPress any key to continue...")
        cv2.waitKey(0)
        cv2.destroyAllWindows()
    
    def _print_restoration_plan(self):
        """Print restoration plan for all frames"""
        print("\nCorruption types and restoration methods:")
        for idx in Config.CORRUPTED_INDICES:
            corruption_type = CorruptionTypeDetector.get_corruption_type(idx)
            method = self._get_restoration_method(idx)
            print(f"  Frame {idx:2d} ({corruption_type:30s}): {method}")
        print("=" * 70 + "\n")
    
    @staticmethod
    def _get_restoration_method(idx: int) -> str:
        """Get restoration method description"""
        if idx == 0:
            return "Keep person from frame 0 + background from frame 1 (2 masks)"
        elif idx in Config.ROTATION_FRAMES:
            return "Auto-rotate + fill black with neighbors"
        elif idx in Config.REPLACE_FRAMES:
            return f"Auto-replace with frame {Config.REPLACE_FRAMES[idx]}"
        elif idx in Config.POSITION_SWAP_FRAMES:
            if idx == 23:
                return "Auto-remove salt noise then swap with frame 14"
            else:
                return "Auto-swap with frame 23"
        elif idx in Config.SALT_FRAMES or idx in Config.PEPPER_FRAMES:
            if idx not in Config.POSITION_SWAP_FRAMES:
                noise_type = "salt" if idx in Config.SALT_FRAMES else "pepper"
                return f"Auto-remove {noise_type} noise only"
        elif idx in Config.LAST_FRAME:
            return "Auto-copy from previous frame"
        elif idx in Config.BLUR_FRAMES:
            return "Mark person + sharpen foreground + average background"
        else:
            return "Mark person + keep foreground + average background"


class SubmissionCreator:
    """Creates submission CSV from restored frames"""
    
    def __init__(self, frames_dir: str = 'frames'):
        self.frames_dir = frames_dir
    
    def create_submission(self, output_csv: str = 'submission.csv') -> pd.DataFrame:
        """Create submission CSV from restored frame files"""
        print("\n" + "=" * 70)
        print("CREATING SUBMISSION CSV")
        print("=" * 70)
        
        final_dict = {"ID": list(range(3 * 1000 * 1000))}
        
        for idx in Config.CORRUPTED_INDICES:
            restored_path = os.path.join(self.frames_dir, f'frame_{idx}_restored.png')
            
            if os.path.exists(restored_path):
                frame = cv2.imread(restored_path)
                arr = np.array(frame, dtype=np.uint8)
                final_dict[f"f{idx}"] = arr.flatten()
                print(f"  Added frame {idx} to submission")
            else:
                print(f"  Warning: Restored frame {idx} not found!")
        
        df = pd.DataFrame(final_dict)
        df.to_csv(output_csv, index=False)
        
        print(f"\n✓ Submission CSV saved: {output_csv}")
        print(f"  Shape: {df.shape}")
        print("=" * 70)
        
        return df


class GIFCreator:
    """Creates GIF from frames"""
    
    def __init__(self, frames_dir: str = 'frames'):
        self.frames_dir = frames_dir
    
    def create_gif(self, output_gif: str = 'restored.gif'):
        """Create final GIF with restored frames"""
        print("\n" + "=" * 70)
        print("CREATING FINAL GIF")
        print("=" * 70)
        
        final_frames = []
        
        for i in range(43):
            if i in Config.CORRUPTED_INDICES:
                frame_path = os.path.join(self.frames_dir, f'frame_{i}_restored.png')
            else:
                frame_path = os.path.join(self.frames_dir, f'frame_{i}.png')
            
            if os.path.exists(frame_path):
                frame = cv2.imread(frame_path)
                frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                final_frames.append(frame_rgb)
                print(f"  Added frame {i}")
        
        imageio.mimsave(output_gif, final_frames, duration=0.1)
        
        print(f"\n✓ Final GIF saved: {output_gif}")
        print(f"  Total frames: {len(final_frames)}")
        print("=" * 70)


class MSECalculator:
    """Calculates MSE between original and restored frames"""
    
    def __init__(self, frames_dir: str = 'frames'):
        self.frames_dir = frames_dir
    
    def compute_mse_all_frames(self) -> List[float]:
        """Compute MSE between original corrupted and restored frames"""
        print("\n" + "=" * 70)
        print("COMPUTING MSE")
        print("=" * 70)
        
        mse_list = []
        
        print(f"{'Frame':<10} {'Type':<30} {'MSE':<15}")
        print("-" * 55)
        
        for idx in Config.CORRUPTED_INDICES:
            corrupted_path = os.path.join(self.frames_dir, f'frame_{idx}.png')
            restored_path = os.path.join(self.frames_dir, f'frame_{idx}_restored.png')
            
            if os.path.exists(corrupted_path) and os.path.exists(restored_path):
                corrupted = cv2.imread(corrupted_path).astype(float)
                restored = cv2.imread(restored_path).astype(float)
                
                mse = np.mean((corrupted - restored) ** 2)
                mse_list.append(mse)
                
                corruption_type = CorruptionTypeDetector.get_corruption_type(idx)
                print(f"{idx:<10} {corruption_type:<30} {mse:<15.4f}")
        
        print("-" * 55)
        print(f"{'TOTAL':<10} {'':<30} {sum(mse_list):<15.4f}")
        print("=" * 70)
        
        return mse_list


class RestorationWorkflow:
    """Main workflow orchestrator"""
    
    def __init__(self, gif_path: str = 'input.gif'):
        self.gif_path = gif_path
        self.frames_dir = 'frames'
        self.masks_dir = 'masks'
        
        self.extractor = FrameExtractor(gif_path, self.frames_dir)
        self.restoration_manager = RestorationManager(self.frames_dir, self.masks_dir)
        self.submission_creator = SubmissionCreator(self.frames_dir)
        self.gif_creator = GIFCreator(self.frames_dir)
        self.mse_calculator = MSECalculator(self.frames_dir)
    
    def run_complete_workflow(self):
        """Execute complete restoration workflow"""
        self._print_header()
        
        print("\n[STEP 1] Extracting frames from GIF...")
        num_frames = self.extractor.extract_all_frames()
        
        print("\n[STEP 2] Restoring all corrupted frames...")
        self._print_restoration_info()
        print("\nPress Enter to continue...")
        input()
        
        restored_frames = self.restoration_manager.restore_all_corrupted_frames()
        
        print("\n[STEP 3] Creating submission CSV...")
        df = self.submission_creator.create_submission('submission.csv')
        
        print("\n[STEP 4] Creating final restored GIF...")
        self.gif_creator.create_gif('restored.gif')
        
        print("\n[STEP 5] Computing MSE scores...")
        mse_list = self.mse_calculator.compute_mse_all_frames()
        
        self._print_summary()
    
    def run_single_frame_restoration(self, frame_idx: int):
        """Restore a single frame for testing"""
        print(f"\n[SINGLE FRAME RESTORATION] Frame {frame_idx}")
        restored = self.restoration_manager._restore_single_frame(frame_idx)
        return restored
    
    @staticmethod
    def _print_header():
        """Print workflow header"""
        print("\n" + "#" * 70)
        print("#" + " " * 68 + "#")
        print("#" + " " * 15 + "HYBRID RESTORATION WORKFLOW" + " " * 27 + "#")
        print("#" + " " * 68 + "#")
        print("#" * 70 + "\n")
    
    @staticmethod
    def _print_restoration_info():
        """Print restoration information"""
        print("\nFrames requiring manual masking:")
        print("  - Frame 0: Mark person in BOTH frame 0 and frame 1")
        print("  - Frame 18: Mark person")
        print("  - Frame 33: Mark person (blur case)")
        print("\nFrames with automatic restoration:")
        print("  - Frames 4, 37: Auto-rotation")
        print("  - Frame 9: Replace with frame 10")
        print("  - Frames 14, 23: Auto-swap positions + remove salt noise from 23")
        print("  - Frame 28: Auto-pepper noise removal")
        print("  - Frame 32: Auto-replace with frame 33")
        print("  - Frame 42: Auto-copy from previous frame")
    
    @staticmethod
    def _print_summary():
        """Print workflow summary"""
        print("\n" + "#" * 70)
        print("#" + " " * 68 + "#")
        print("#" + " " * 20 + "🎉 WORKFLOW COMPLETE! 🎉" + " " * 23 + "#")
        print("#" + " " * 68 + "#")
        print("#" * 70)
        print("\nGenerated files:")
        print("  📁 frames/         - All extracted and restored frames")
        print("  📁 masks/          - Foreground masks")
        print("  📄 submission.csv  - Ready for Kaggle submission")
        print("  🎬 restored.gif    - Final restored video")
        print("\nYou can now submit 'submission.csv' to Kaggle!")
        print("#" * 70 + "\n")


def segment_all_corrupted_frames():
    """Interactively segment ALL corrupted frames"""
    segmenter = ManualSegmentationWorkflow()
    
    if not os.path.exists('frames/frame_0.png'):
        print("Extracting frames first...")
        extractor = FrameExtractor('input.gif', 'frames')
        extractor.extract_all_frames()
    
    needs_masks = segmenter.list_frames_needing_masks()
    
    if needs_masks:
        response = input(f"\nSegment {len(needs_masks)} frames now? (y/n): ").lower()
        if response == 'y':
            segmenter.segment_multiple_frames(needs_masks)
    else:
        print("\n✅ All frames already have masks!")


def segment_specific_frames():
    """Segment only specific frames you choose"""
    segmenter = ManualSegmentationWorkflow()
    
    if not os.path.exists('frames/frame_0.png'):
        print("Extracting frames first...")
        extractor = FrameExtractor('input.gif', 'frames')
        extractor.extract_all_frames()
    
    frames_to_segment = [0, 4, 9, 14, 18, 23, 28, 32, 37, 42]
    
    segmenter.segment_multiple_frames(frames_to_segment)


def resegment_frame(frame_num: int):
    """Re-do segmentation for a specific frame"""
    segmenter = ManualSegmentationWorkflow()
    segmenter.segment_frame(frame_num, force_new=True)


def main_with_manual_segmentation():
    """Main workflow with manual segmentation option"""
    
    print("\n" + "#"*70)
    print("# FRAME RESTORATION WITH MANUAL SEGMENTATION")
    print("#"*70)
    print("\nOptions:")
    print("  1. Segment all corrupted frames manually")
    print("  2. Segment specific frames only")
    print("  3. Check mask status")
    print("  4. Run full restoration (after segmentation)")
    print("  5. Exit")
    
    choice = input("\nSelect option (1-5): ").strip()
    
    segmenter = ManualSegmentationWorkflow()
    
    if choice == '1':
        segment_all_corrupted_frames()
    elif choice == '2':
        frames_str = input("Enter frame numbers (comma-separated, e.g., 0,18,33): ")
        frames = [int(x.strip()) for x in frames_str.split(',')]
        segmenter.segment_multiple_frames(frames)
    elif choice == '3':
        segmenter.list_frames_needing_masks()
    elif choice == '4':
        workflow = RestorationWorkflow('input.gif')
        workflow.run_complete_workflow()
    else:
        print("Exiting...")


def test_masking():
    """Test the masking functionality independently"""
    print("\n" + "=" * 80)
    print("TESTING MASK CREATION TOOL")
    print("=" * 80)
    
    if not os.path.exists('frames/frame_0.png'):
        print("❌ Error: Please extract frames first!")
        print("   Run: extractor = FrameExtractor('input.gif'); extractor.extract_all_frames()")
        return
    
    test_frame = cv2.imread('frames/frame_0.png')
    
    if test_frame is None:
        print("❌ Error: Could not load test frame!")
        return
    
    print("✓ Loaded test frame (frame_0.png)")
    print("\nOpening mask creation window...")
    
    mask_creator = MaskCreator()
    mask = mask_creator.create_mask_interactive(test_frame, "TEST MASKING - Frame 0")
    
    if mask is not None:
        os.makedirs('masks', exist_ok=True)
        cv2.imwrite('masks/test_mask.png', mask)
        print("\n✅ Test mask saved to: masks/test_mask.png")
        
        mask_gray = cv2.cvtColor(mask, cv2.COLOR_BGR2GRAY)
        result = np.where(mask_gray[:, :, np.newaxis] == 255, test_frame, 
                         np.zeros_like(test_frame))
        
        cv2.imshow("Masked Result", result)
        print("\nShowing masked result. Press any key to close...")
        cv2.waitKey(0)
        cv2.destroyAllWindows()
    else:
        print("\n❌ Masking cancelled or failed")


def main():
    """Main entry point - original workflow"""
    workflow = RestorationWorkflow('input.gif')
    workflow.run_complete_workflow()
import cv2
import numpy as np
import imageio
import os
import pandas as pd
from scipy.ndimage import median_filter

# Global variables
CORRUPTED_INDICES = [0, 4, 9, 14, 18, 23, 28, 32, 37, 42]
FRAME_SIZE = (1000, 1000, 3)

# Frame-specific corruption types
ROTATION_FRAMES = {4: 135, 37: 60}  # frame_index: rotation_angle
SALT_FRAMES = [23]  # Frame 23 has salt noise
PEPPER_FRAMES = [28]  # Frame 28 has pepper noise
BLUR_FRAMES = [33]  # Changed from 32 to 33
POSITION_SWAP_FRAMES = {14: 23, 23: 14}  # Swapped positions
RED_NOISE_FRAMES = []  # No red noise anymore
LAST_FRAME = [42]  # Just use previous frame
FIRST_FRAME = [0]  # Special case: needs both masks
REPLACE_FRAMES = {9: 10, 32: 33}  # Frame 9 -> 10, Frame 32 -> 33

# ========== STEP 1: EXTRACT ALL FRAMES FROM GIF ==========
def extract_all_frames(gif_path='input.gif', output_dir='frames'):
    """
    Extract all frames from GIF and save as individual PNG files
    """
    os.makedirs(output_dir, exist_ok=True)
    
    print("="*70)
    print("EXTRACTING FRAMES FROM GIF")
    print("="*70)
    
    frames = imageio.mimread(gif_path)
    print(f"Total frames found: {len(frames)}")
    
    for i, frame in enumerate(frames):
        frame_bgr = cv2.cvtColor(frame, cv2.COLOR_RGB2BGR)
        frame_path = os.path.join(output_dir, f'frame_{i}.png')
        cv2.imwrite(frame_path, frame_bgr)
        
        if i in CORRUPTED_INDICES:
            corruption_type = get_corruption_type(i)
            print(f"  Saved: frame_{i}.png [CORRUPTED - {corruption_type}]")
        else:
            print(f"  Saved: frame_{i}.png")
    
    print("="*70)
    print(f"✓ All {len(frames)} frames extracted to '{output_dir}/' directory")
    print("="*70)
    
    return len(frames)


def get_corruption_type(frame_idx):
    """Identify corruption type for a frame"""
    if frame_idx in ROTATION_FRAMES:
        return f"ROTATION ({ROTATION_FRAMES[frame_idx]}°)"
    elif frame_idx == 23:
        return "SWAPPED WITH 14 + SALT NOISE"
    elif frame_idx == 14:
        return "SWAPPED WITH 23"
    elif frame_idx in PEPPER_FRAMES:
        return "PEPPER NOISE"
    elif frame_idx in REPLACE_FRAMES:
        return f"REPLACE WITH FRAME {REPLACE_FRAMES[frame_idx]}"
    elif frame_idx in BLUR_FRAMES:
        return "BLUR"
    elif frame_idx in LAST_FRAME:
        return "COPY PREVIOUS"
    elif frame_idx in FIRST_FRAME:
        return "SPECIAL - FOREGROUND + BACKGROUND"
    else:
        return "FOREGROUND/BACKGROUND"


# ========== SALT AND PEPPER NOISE REMOVAL ==========
def detect_salt_pepper_noise(img, threshold_low=10, threshold_high=245):
    """
    Detect salt (white) and pepper (black) noise pixels
    Returns a binary mask where 1 indicates noise pixel
    """
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
    # Detect pepper noise (very dark pixels)
    pepper_mask = gray < threshold_low
    
    # Detect salt noise (very bright pixels)
    salt_mask = gray > threshold_high
    
    # Combine masks
    noise_mask = np.logical_or(salt_mask, pepper_mask)
    
    return noise_mask


def restore_salt_pepper_precise(frame_idx, frames_dir='frames'):
    """
    Restore salt/pepper noise by replacing ONLY noise pixels using median of neighbors
    """
    print(f"\nRestoring salt/pepper noise frame {frame_idx}")
    
    # Load current frame
    img_current = cv2.imread(os.path.join(frames_dir, f'frame_{frame_idx}.png'))
    
    # Detect noise pixels
    noise_mask = detect_salt_pepper_noise(img_current)
    
    # Create a copy for restoration
    restored = img_current.copy()
    
    # For each noisy pixel, replace with median of 3x3 neighborhood
    for c in range(3):  # For each color channel
        channel = img_current[:, :, c]
        
        # Apply median filter only to the channel
        median_filtered = cv2.medianBlur(channel, 3)
        
        # Replace only noise pixels
        restored[:, :, c] = np.where(noise_mask, median_filtered, channel)
    
    print(f"  Replaced {np.sum(noise_mask)} noise pixels")
    
    return restored


# ========== POSITION SWAP RESTORATION ==========
def restore_swapped_frames(frame_idx, frames_dir='frames'):
    """
    Restore frames where positions are swapped (14 and 23)
    Frame 14: Has person from frame 23
    Frame 23: Has person from frame 14 + salt noise
    
    Strategy: 
    - For frame 14: Use frame 23 as-is (it will be cleaned when processing frame 23)
    - For frame 23: Remove salt noise first, then swap
    """
    print(f"\nRestoring swapped frame {frame_idx}")
    
    if frame_idx == 14:
        # Frame 14 has person from 23
        # We'll use the cleaned version of frame 23
        img_23 = cv2.imread(os.path.join(frames_dir, 'frame_23.png'))
        
        # Check if frame 23 is already restored
        restored_23_path = os.path.join(frames_dir, 'frame_23_restored.png')
        if os.path.exists(restored_23_path):
            print(f"  Using already restored frame 23")
            return cv2.imread(restored_23_path)
        else:
            # Use frame 23 as-is (will be swapped back when frame 23 is processed)
            print(f"  Using frame 23 content (will be finalized after frame 23 restoration)")
            return img_23
        
    elif frame_idx == 23:
        # Frame 23 has person from 14 + salt noise
        # First remove salt noise, then use frame 14's content
        img_23_corrupted = cv2.imread(os.path.join(frames_dir, 'frame_23.png'))
        
        # Remove salt noise
        denoised = restore_salt_pepper_precise(23, frames_dir)
        
        print(f"  Removed salt noise and using correct content")
        return denoised


# ========== ROTATION RESTORATION ==========
def restore_rotation_frame(frame_idx, angle, frames_dir='frames'):
    """
    Restore rotated frame by rotating back and filling black parts with averaged neighbors
    """
    print(f"\nRestoring rotation frame {frame_idx} (angle: {angle}°)")
    
    img_current = cv2.imread(os.path.join(frames_dir, f'frame_{frame_idx}.png'))
    
    prev_idx, next_idx = get_neighbor_indices(frame_idx)
    img_before, img_after = load_neighbor_frames(prev_idx, next_idx, frames_dir)
    
    # Rotate back
    height, width = img_current.shape[:2]
    center = (width // 2, height // 2)
    rotation_matrix = cv2.getRotationMatrix2D(center, -angle, 1.0)
    rotated = cv2.warpAffine(img_current, rotation_matrix, (width, height))
    
    # Create mask for black pixels
    gray = cv2.cvtColor(rotated, cv2.COLOR_BGR2GRAY)
    _, black_mask = cv2.threshold(gray, 1, 255, cv2.THRESH_BINARY_INV)
    
    # Average neighbors for background fill
    avg_background = cv2.addWeighted(img_before, 0.5, img_after, 0.5, 0)
    
    # Fill black parts with averaged background
    black_mask_3ch = cv2.cvtColor(black_mask, cv2.COLOR_GRAY2BGR)
    restored = np.where(black_mask_3ch == 255, avg_background, rotated)
    
    return restored


# ========== INTERACTIVE MASK CREATION ==========
def create_mask_interactive(image, window_title):
    """Create foreground mask interactively"""
    global drawing, erasing, mask, brush_size
    drawing, erasing = False, False
    brush_size = 15
    mask = np.zeros((image.shape[0], image.shape[1], 3), dtype=np.uint8)
    cv2.namedWindow(window_title, cv2.WINDOW_NORMAL)
    
    def mouse_callback(event, x, y, flags, param):
        global drawing, erasing, mask, brush_size
        if event == cv2.EVENT_LBUTTONDOWN:
            drawing = True
            cv2.circle(mask, (x, y), brush_size, (255, 255, 255), -1)
        elif event == cv2.EVENT_RBUTTONDOWN:
            erasing = True
            cv2.circle(mask, (x, y), brush_size, (0, 0, 0), -1)
        elif event == cv2.EVENT_MOUSEMOVE:
            if drawing:
                cv2.circle(mask, (x, y), brush_size, (255, 255, 255), -1)
            elif erasing:
                cv2.circle(mask, (x, y), brush_size, (0, 0, 0), -1)
        elif event == cv2.EVENT_LBUTTONUP:
            drawing = False
        elif event == cv2.EVENT_RBUTTONUP:
            erasing = False
    
    cv2.setMouseCallback(window_title, mouse_callback)
    
    print(f"\n{'='*70}")
    print(f"🖌  NOW EDITING: {window_title}")
    print(f"{'='*70}")
    print("Controls:")
    print("  LEFT CLICK & DRAG  : Mark as Foreground (Person)")
    print("  RIGHT CLICK & DRAG : Mark as Background")
    print("  '+' or '='         : Increase brush size")
    print("  '-'                : Decrease brush size")
    print("  's'                : SAVE mask and continue")
    print("  'q'                : QUIT without saving")
    print(f"{'='*70}\n")

    while True:
        mask_gray_for_overlay = cv2.cvtColor(mask, cv2.COLOR_BGR2GRAY)
        overlay_green = np.zeros_like(image, dtype=np.uint8)
        overlay_green[mask_gray_for_overlay == 255] = (0, 255, 0)
        display_image = cv2.addWeighted(image, 1, overlay_green, 0.4, 0)
        cv2.putText(display_image, f"Brush Size: {brush_size}", (10, 30),
                   cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)
        cv2.imshow(window_title, display_image)
        
        key = cv2.waitKey(1) & 0xFF
        if key == ord('+') or key == ord('='):
            brush_size += 2
        elif key == ord('-') and brush_size > 1:
            brush_size -= 2
        elif key == ord('q'):
            cv2.destroyAllWindows()
            return None
        elif key == ord('s'):
            break
    
    cv2.destroyAllWindows()
    return mask


# ========== SHARPENING FILTER ==========
def sharpen_image(img):
    """Apply sharpening to image"""
    kernel = np.array([[-1, -1, -1],
                       [-1,  9, -1],
                       [-1, -1, -1]])
    sharpened = cv2.filter2D(img, -1, kernel)
    return sharpened


# ========== NEIGHBOR FRAME UTILITIES ==========
def get_neighbor_indices(frame_idx):
    """Get previous and next frame indices, skipping corrupted ones"""
    prev_idx = frame_idx - 1
    next_idx = frame_idx + 1
    
    while prev_idx in CORRUPTED_INDICES and prev_idx >= 0:
        prev_idx -= 1
    
    while next_idx in CORRUPTED_INDICES and next_idx <= 42:
        next_idx += 1
    
    if prev_idx < 0:
        prev_idx = None
    if next_idx > 42:
        next_idx = None
    
    return prev_idx, next_idx


def load_neighbor_frames(prev_idx, next_idx, frames_dir):
    """Load neighbor frames, handling missing frames"""
    img_before = None
    img_after = None
    
    if prev_idx is not None:
        img_before = cv2.imread(os.path.join(frames_dir, f'frame_{prev_idx}.png'))
    
    if next_idx is not None:
        img_after = cv2.imread(os.path.join(frames_dir, f'frame_{next_idx}.png'))
    
    return img_before, img_after


# ========== FRAME 0 SPECIAL RESTORATION ==========
def restore_frame_0_special(frames_dir='frames', masks_dir='masks'):
    """
    Special restoration for frame 0:
    - Keep foreground (person) from frame 0
    - Replace background with frame 1 background
    - Where person was removed from frame 1, fill with frame 0 background
    """
    print("\nRestoring Frame 0 - Special Case")
    print("Need to mark person in BOTH frame 0 and frame 1")
    
    img_frame_0 = cv2.imread(os.path.join(frames_dir, 'frame_0.png'))
    img_frame_1 = cv2.imread(os.path.join(frames_dir, 'frame_1.png'))
    
    # Get mask for frame 0 (person to keep)
    mask_0_path = os.path.join(masks_dir, 'mask_frame_0.png')
    if os.path.exists(mask_0_path):
        print("Found existing mask for frame 0")
        fg_mask_0 = cv2.imread(mask_0_path)
    else:
        print("\n--- STEP 1: Mark the PERSON in Frame 0 (to keep) ---")
        fg_mask_0 = create_mask_interactive(img_frame_0, "Mark Person in Frame 0 (to KEEP)")
        if fg_mask_0 is None:
            return None
        os.makedirs(masks_dir, exist_ok=True)
        cv2.imwrite(mask_0_path, fg_mask_0)
        print(f"✓ Mask saved: {mask_0_path}")
    
    # Get mask for frame 1 (person to remove)
    mask_1_path = os.path.join(masks_dir, 'mask_frame_1.png')
    if os.path.exists(mask_1_path):
        print("Found existing mask for frame 1")
        fg_mask_1 = cv2.imread(mask_1_path)
    else:
        print("\n--- STEP 2: Mark the PERSON in Frame 1 (to remove) ---")
        fg_mask_1 = create_mask_interactive(img_frame_1, "Mark Person in Frame 1 (to REMOVE)")
        if fg_mask_1 is None:
            return None
        cv2.imwrite(mask_1_path, fg_mask_1)
        print(f"✓ Mask saved: {mask_1_path}")
    
    # Convert masks to grayscale for easier processing
    fg_mask_0_gray = cv2.cvtColor(fg_mask_0, cv2.COLOR_BGR2GRAY)
    fg_mask_1_gray = cv2.cvtColor(fg_mask_1, cv2.COLOR_BGR2GRAY)
    
    # Create final restored frame
    # Priority: Person from frame 0 > Background from frame 1 > Background from frame 0
    restored = img_frame_1.copy()
    
    # Where person in frame 1 was marked, replace with background from frame 0
    # (This reduces MSE by filling the removed area)
    restored = np.where(fg_mask_1_gray[:, :, np.newaxis] == 255, img_frame_0, restored)
    
    # Where person in frame 0 is marked, use frame 0 person (override everything)
    restored = np.where(fg_mask_0_gray[:, :, np.newaxis] == 255, img_frame_0, restored)
    
    print("✓ Combined: Person from frame 0 + Background from frame 1 + Fill removed areas with frame 0")
    
    return restored


# ========== HYBRID RESTORATION FOR SINGLE FRAME ==========
def restore_single_frame_hybrid(frame_idx, frames_dir='frames', masks_dir='masks'):
    """
    Restore a single corrupted frame based on its corruption type
    """
    print("\n" + "="*70)
    print(f"RESTORING FRAME {frame_idx} - {get_corruption_type(frame_idx)}")
    print("="*70)
    
    # Frame 0: Special case with two masks
    if frame_idx == 0:
        restored = restore_frame_0_special(frames_dir, masks_dir)
        if restored is None:
            return None
    
    # Handle rotation frames
    elif frame_idx in ROTATION_FRAMES:
        angle = ROTATION_FRAMES[frame_idx]
        restored = restore_rotation_frame(frame_idx, angle, frames_dir)
    
    # Handle swapped frames (14 and 23)
    elif frame_idx in POSITION_SWAP_FRAMES:
        restored = restore_swapped_frames(frame_idx, frames_dir)
    
    # Handle salt/pepper noise only frames
    elif frame_idx in SALT_FRAMES or frame_idx in PEPPER_FRAMES:
        if frame_idx not in POSITION_SWAP_FRAMES:
            restored = restore_salt_pepper_precise(frame_idx, frames_dir)
    
    # Handle replace frames (9->10, 32->33)
    elif frame_idx in REPLACE_FRAMES:
        replace_with = REPLACE_FRAMES[frame_idx]
        restored = cv2.imread(os.path.join(frames_dir, f'frame_{replace_with}.png'))
        print(f"Replaced with frame {replace_with}")
    
    # Handle last frame (copy previous)
    elif frame_idx in LAST_FRAME:
        prev_idx, _ = get_neighbor_indices(frame_idx)
        restored = cv2.imread(os.path.join(frames_dir, f'frame_{prev_idx}.png'))
        print(f"Copied from frame {prev_idx}")
    
    # Blur frames and other frames requiring masking (18, 33)
    else:
        prev_idx, next_idx = get_neighbor_indices(frame_idx)
        img_before, img_after = load_neighbor_frames(prev_idx, next_idx, frames_dir)
        img_current = cv2.imread(os.path.join(frames_dir, f'frame_{frame_idx}.png'))
        
        # Handle case when only one neighbor is available
        if img_before is None and img_after is not None:
            print(f"Only next frame {next_idx} available, using it for background")
            restored_background = img_after
        elif img_after is None and img_before is not None:
            print(f"Only previous frame {prev_idx} available, using it for background")
            restored_background = img_before
        elif img_before is not None and img_after is not None:
            print(f"Using neighbors: frame_{prev_idx} and frame_{next_idx}")
            restored_background = cv2.addWeighted(img_before, 0.5, img_after, 0.5, 0)
        else:
            print("Error: No neighbor frames available!")
            return None
        
        # Check if mask already exists
        mask_path = os.path.join(masks_dir, f'mask_frame_{frame_idx}.png')
        
        if os.path.exists(mask_path):
            print(f"Found existing mask: {mask_path}")
            fg_mask = cv2.imread(mask_path)
        else:
            print("No existing mask found. Please create one...")
            fg_mask = create_mask_interactive(img_current, f"Mark Person in Frame {frame_idx}")
            
            if fg_mask is None:
                print("Operation cancelled by user.")
                return None
            
            os.makedirs(masks_dir, exist_ok=True)
            cv2.imwrite(mask_path, fg_mask)
            print(f"✓ Mask saved: {mask_path}")
        
        # For blur frames, sharpen the foreground
        if frame_idx in BLUR_FRAMES:
            print("Applying sharpening to foreground...")
            foreground = sharpen_image(img_current)
        else:
            foreground = img_current
        
        # Assemble final image
        restored = np.where(fg_mask == 255, foreground, restored_background)
    
    # Save result
    output_filename = os.path.join(frames_dir, f'frame_{frame_idx}_restored.png')
    cv2.imwrite(output_filename, restored)
    print(f"\n✅ Frame {frame_idx} restoration complete!")
    print(f"   Saved as: {output_filename}")
    
    # Display result
    cv2.namedWindow(f'Final Restored Frame {frame_idx}', cv2.WINDOW_NORMAL)
    cv2.imshow(f'Final Restored Frame {frame_idx}', restored)
    print("\nPress any key to continue...")
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    
    return restored


# ========== RESTORE ALL CORRUPTED FRAMES ==========
def restore_all_corrupted_frames(frames_dir='frames', masks_dir='masks'):
    """
    Restore all corrupted frames one by one
    """
    print("\n" + "="*70)
    print("RESTORING ALL CORRUPTED FRAMES")
    print("="*70)
    print(f"Corrupted frames to restore: {CORRUPTED_INDICES}")
    
    print("\nCorruption types and restoration methods:")
    for idx in CORRUPTED_INDICES:
        corruption_type = get_corruption_type(idx)
        if idx == 0:
            method = "Keep person from frame 0 + background from frame 1 (2 masks)"
        elif idx in ROTATION_FRAMES:
            method = "Auto-rotate + fill black with neighbors"
        elif idx in REPLACE_FRAMES:
            method = f"Auto-replace with frame {REPLACE_FRAMES[idx]}"
        elif idx in POSITION_SWAP_FRAMES:
            if idx == 23:
                method = "Auto-remove salt noise then swap with frame 14"
            else:  # idx == 14
                method = "Auto-swap with frame 23"
        elif idx in SALT_FRAMES or idx in PEPPER_FRAMES:
            if idx not in POSITION_SWAP_FRAMES:
                noise_type = "salt" if idx in SALT_FRAMES else "pepper"
                method = f"Auto-remove {noise_type} noise only"
        elif idx in LAST_FRAME:
            method = "Auto-copy from previous frame"
        elif idx in BLUR_FRAMES:
            method = "Mark person + sharpen foreground + average background"
        else:
            method = "Mark person + keep foreground + average background"
        
        print(f"  Frame {idx:2d} ({corruption_type:30s}): {method}")
    
    print("="*70 + "\n")
    
    restored_frames = {}
    
    for idx in CORRUPTED_INDICES:
        print(f"\n{'#'*70}")
        print(f"# Processing Frame {idx} ({CORRUPTED_INDICES.index(idx) + 1}/{len(CORRUPTED_INDICES)})")
        print(f"{'#'*70}")
        
        restored = restore_single_frame_hybrid(idx, frames_dir, masks_dir)
        
        if restored is not None:
            restored_frames[idx] = restored
            print(f"✓ Frame {idx} completed")
        else:
            print(f"✗ Frame {idx} skipped")
    
    print("\n" + "="*70)
    print(f"✓ RESTORATION COMPLETE!")
    print(f"  Restored {len(restored_frames)}/{len(CORRUPTED_INDICES)} frames")
    print("="*70)
    
    return restored_frames


# ========== CREATE SUBMISSION CSV ==========
def create_submission_from_frames(frames_dir='frames', output_csv='submission.csv'):
    """
    Create submission CSV from restored frame files
    """
    print("\n" + "="*70)
    print("CREATING SUBMISSION CSV")
    print("="*70)
    
    final_dict = {"ID": list(range(3 * 1000 * 1000))}
    
    for idx in CORRUPTED_INDICES:
        restored_path = os.path.join(frames_dir, f'frame_{idx}_restored.png')
        
        if os.path.exists(restored_path):
            frame = cv2.imread(restored_path)
            arr = np.array(frame, dtype=np.uint8)
            final_dict[f"f{idx}"] = arr.flatten()
            print(f"  Added frame {idx} to submission")
        else:
            print(f"  Warning: Restored frame {idx} not found!")
    
    df = pd.DataFrame(final_dict)
    df.to_csv(output_csv, index=False)
    
    print(f"\n✓ Submission CSV saved: {output_csv}")
    print(f"  Shape: {df.shape}")
    print("="*70)
    
    return df


# ========== CREATE FINAL GIF ==========
def create_final_gif(frames_dir='frames', output_gif='restored.gif'):
    """
    Create final GIF with restored frames
    """
    print("\n" + "="*70)
    print("CREATING FINAL GIF")
    print("="*70)
    
    final_frames = []
    
    for i in range(43):
        if i in CORRUPTED_INDICES:
            frame_path = os.path.join(frames_dir, f'frame_{i}_restored.png')
        else:
            frame_path = os.path.join(frames_dir, f'frame_{i}.png')
        
        if os.path.exists(frame_path):
            frame = cv2.imread(frame_path)
            frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            final_frames.append(frame_rgb)
            print(f"  Added frame {i}")
    
    imageio.mimsave(output_gif, final_frames, duration=0.1)
    
    print(f"\n✓ Final GIF saved: {output_gif}")
    print(f"  Total frames: {len(final_frames)}")
    print("="*70)


# ========== COMPUTE MSE ==========
def compute_mse_all_frames(frames_dir='frames'):
    """
    Compute MSE between original corrupted and restored frames
    """
    print("\n" + "="*70)
    print("COMPUTING MSE")
    print("="*70)
    
    mse_list = []
    
    print(f"{'Frame':<10} {'Type':<30} {'MSE':<15}")
    print("-"*55)
    
    for idx in CORRUPTED_INDICES:
        corrupted_path = os.path.join(frames_dir, f'frame_{idx}.png')
        restored_path = os.path.join(frames_dir, f'frame_{idx}_restored.png')
        
        if os.path.exists(corrupted_path) and os.path.exists(restored_path):
            corrupted = cv2.imread(corrupted_path).astype(float)
            restored = cv2.imread(restored_path).astype(float)
            
            mse = np.mean((corrupted - restored) ** 2)
            mse_list.append(mse)
            
            corruption_type = get_corruption_type(idx)
            print(f"{idx:<10} {corruption_type:<30} {mse:<15.4f}")
    
    print("-"*55)
    print(f"{'TOTAL':<10} {'':<30} {sum(mse_list):<15.4f}")
    print("="*70)
    
    return mse_list


# ========== MAIN WORKFLOW ==========
def gif_submit(gif_path='input.gif'):
    """
    Complete workflow: Extract → Restore → Create Submission
    """
    print("\n" + "#"*70)
    print("#" + " "*68 + "#")
    print("#" + " "*15 + "HYBRID RESTORATION WORKFLOW" + " "*27 + "#")
    print("#" + " "*68 + "#")
    print("#"*70 + "\n")
    
    # Step 1: Extract all frames
    print("\n[STEP 1] Extracting frames from GIF...")
    num_frames = extract_all_frames(gif_path, 'frames')
    
    # Step 2: Restore all corrupted frames
    print("\n[STEP 2] Restoring all corrupted frames...")
    print("\nFrames requiring manual masking:")
    print("  - Frame 0: Mark person in BOTH frame 0 and frame 1")
    print("  - Frame 18: Mark person and reduce the red noise")
    print("\nFrames with automatic restoration:")
    print("  - Frames 4, 37: Auto-rotation")
    print("  - Frame 9: Replace with frame 42 and reduce the noise")
    print("  - Frames 14, 23: Auto-swap positions + remove salt noise from 23")
    print("  - Frame 28: Auto-pepper noise removal")
    print("  - Frame 32: Auto-replace with frame 33")
    print("  - Frame 42: Replace with frame 9 and reduce the blur")
    print("\nPress Enter to continue...")
    input()
    
    restored_frames = restore_all_corrupted_frames('frames', 'masks')
    
    # Step 3: Create submission CSV
    print("\n[STEP 3] Creating submission CSV...")
    df = create_submission_from_frames('frames', 'submission.csv')
    
    # Step 4: Create final GIF
    print("\n[STEP 4] Creating final restored GIF...")
    create_final_gif('frames', 'restored.gif')
    
    # Step 5: Compute MSE
    print("\n[STEP 5] Computing MSE scores...")
    mse_list = compute_mse_all_frames('frames')
    
    # Final summary
    print("\n" + "#"*70)
    print("#" + " "*68 + "#")
    print("#" + " "*20 + "🎉 WORKFLOW COMPLETE! 🎉" + " "*23 + "#")
    print("#" + " "*68 + "#")
    print("#"*70)
    print("\nGenerated files:")
    print("  📁 frames/         - All extracted and restored frames")
    print("  📁 masks/          - Foreground masks")
    print("  📄 submission.csv  - Ready for Kaggle submission")
    print("  🎬 restored.gif    - Final restored video")
    print("\nYou can now submit 'submission.csv' to Kaggle!")
    print("#"*70 + "\n")




if __name__ == '__main__':
    # CHOOSE ONE OF THESE OPTIONS:
    
    # Option 1: Interactive menu with manual segmentation
    main_with_manual_segmentation()
    
    # Option 2: Segment all corrupted frames first, then restore
    # segment_all_corrupted_frames()
    gif_submit()
    
    # Option 3: Run original full workflow
    # main()
    
    # Option 4: Test masking tool only
    # test_masking()
    
    # Option 5: Segment specific frame
    # segmenter = ManualSegmentationWorkflow()
    # segmenter.segment_frame(0)
    
    # Option 6: Extract frames only
    # extractor = FrameExtractor('input.gif', 'frames')
    # extractor.extract_all_frames()


######################################################################
# FRAME RESTORATION WITH MANUAL SEGMENTATION
######################################################################

Options:
  1. Segment all corrupted frames manually
  2. Segment specific frames only
  3. Check mask status
  4. Run full restoration (after segmentation)
  5. Exit

MASK STATUS FOR CORRUPTED FRAMES
Frame      Has Mask?       Corruption Type               
----------------------------------------------------------------------
0          ✓ Yes           SPECIAL - FOREGROUND + BACKGROUND
4          ✓ Yes           ROTATION (135°)               
9          ✓ Yes           REPLACE WITH FRAME 10         
14         ✓ Yes           SWAPPED WITH 23               
18         ✓ Yes           FOREGROUND/BACKGROUND         
23         ✓ Yes           SWAPPED WITH 14 + SALT NOISE  
28         ✓ Yes           PEPPER NOISE                  
32         ✓ Yes           REPLACE WITH FRAME 33         
37         ✓ Ye